In [ ]:
import requests
import pandas as pd

def extract_hunter_leads(hunter_response):
    """
    Extract structured lead data from Hunter.io domain search response
    """
    data = hunter_response.get("data", {})
    emails = data.get("emails", [])

    company_name = data.get("organization")
    website = data.get("domain")

    leads = []

    for e in emails:
        lead = {
            "company_name": company_name,
            "website": website,
            "full_name": f"{e.get('first_name', '')} {e.get('last_name', '')}".strip(),
            "email": e.get("value"),
            "position": e.get("position"),
            "seniority": e.get("seniority"),
            "department": e.get("department"),
            "linkedin_url": e.get("linkedin"),
            "twitter_url": e.get("twitter"),
            "hunter_confidence": e.get("confidence")
        }

        leads.append(lead)

    return leads

API_KEY = ""
BASE_URL = "https://api.hunter.io/v2"
WEBSITE = "arcare.com.au"

url = f"{BASE_URL}/domain-search"
params = {
    "domain": WEBSITE,
    "limit": 10,
    "api_key": API_KEY
}

response = requests.get(url, params=params)
data = response.json()

leads = extract_hunter_leads(data)

for lead in leads:
    print(lead)

df = pd.DataFrame(leads)
df.to_csv(f"{WEBSITE}.csv", index=False)

{'company_name': 'Arcare Aged Care', 'website': 'arcare.com.au', 'full_name': 'Andrew Buckle', 'email': 'andrew.buckle@arcare.com.au', 'position': 'Executive Chef', 'seniority': 'executive', 'department': 'operations', 'linkedin_url': 'https://www.linkedin.com/in/andrew-buckle-bbb46661', 'twitter_url': None, 'hunter_confidence': 99}
{'company_name': 'Arcare Aged Care', 'website': 'arcare.com.au', 'full_name': 'Chris Hilton', 'email': 'chris.hilton@arcare.com.au', 'position': 'Chief Financial Officer', 'seniority': 'executive', 'department': 'finance', 'linkedin_url': 'https://www.linkedin.com/in/chris-hilton-0a2aa818', 'twitter_url': None, 'hunter_confidence': 99}
{'company_name': 'Arcare Aged Care', 'website': 'arcare.com.au', 'full_name': 'Colin Singh', 'email': 'colin.singh@arcare.com.au', 'position': 'CEO', 'seniority': 'executive', 'department': 'executive', 'linkedin_url': 'https://www.linkedin.com/in/colin-singh-462a8217', 'twitter_url': None, 'hunter_confidence': 98}
{'company_

In [ ]:
domains = [
    "opalhealthcare.com.au",
    "boltonclarke.com.au",
    "estiahealth.com.au",
    "uniting.org",
    "calvarycare.org.au",
    "arcare.com.au",
    "bluecare.org.au",
    "baptistcare.org.au",
    "australianunity.com.au",
    "aegiscare.com.au",
    "svha.org.au",
    "allity.com.au",
    "mercyhealth.com.au",
    "rsllifecare.org.au",
    "irt.org.au",
    "infinitecare.com.au",
    "hammondcare.com.au",
    "pach.org.au",
    "anglicare.org.au",
    "southerncrosscare.com.au",
    "mckenzieacg.com",
    "hallprior.com.au",
    "fivegoodfriends.com.au",
    "kincare.com.au",
    "silverchain.org.au",
    "minda.asn.au",
    "dovida.com.au",
    "trilogycare.com.au",
    "tlcagedcare.com.au",
    "resthaven.asn.au",
    "ech.asn.au",
    "benetas.com.au",
    "cofcqld.com.au",
    "vmch.com.au",
    "juniper.org.au",
    "luson.com.au",
    "signaturecare.com.au",
    "domainagedcare.com.au",
    "mayflower.org.au",
    "warrigal.com.au",
    "caringchoice.com.au",
    "ozcare.org.au",
    "csaust.org.au",
    "aveo.com.au",
    "catholichomes.com",
    "freshwaterbayagedcare.com.au",
    "brightwatergroup.com",
    "apexcare.com.au",
    "sigmahealthcare.com.au",
    "ebosgroup.com",
    "telixpharma.com",
    "maynepharma.com",
    "mesoblast.com",
    "neurenpharma.com",
    "starpharma.com",
    "clinuvel.com",
    "immutep.com",
    "resmed.com",
    "cochlear.com",
    "promedicus.com",
    "polynovo.com",
    "trajanscimed.com",
    "paragoncare.com.au",
    "device.com.au",
    "healthscope.com.au",
    "forhealth.com.au",
    "monashivfgroup.com.au",
    "ipn.com.au",
    "4dmedical.com",
    "adalta.com.au",
    "alcidion.com",
    "ampliatx.com",
    "artrya.com",
    "botanixpharma.com",
    "claritypharmaceuticals.com",
    "cogstate.com",
    "dimerix.com",
    "imugene.com",
    "lifehealthcare.com.au",
    "lumosdiagnostics.com",
    "medicaldev.com",
    "microba.com",
    "opthea.com",
    "orthocell.com.au",
    "proteomics.com.au",
    "recce.com.au",
    "sdi.com.au",
    "telstrahealth.com",
    "rhinomed.global",
    "volparahealth.com",
    "compumedics.com",
    "cardiex.com",
    "medadvisor.com.au",
    "global-health.com",
    "hevert.com.au",
    "paradigmbiopharma.com",
    "actinogen.com.au",
    "biotron.com.au",
    "geneticsignatures.com",
    "oncosil.com",
    "ausbiotech.org",
    "oventusmedical.com",
    "virtushealth.com.au",
    "healthecare.com.au",
    "greencross.com.au",
    "nationaldentalcare.com.au",
    "audika.com.au",
    "connecthearing.com.au",
    "visioneyeinstitute.com.au",
    "cryosite.com",
    "laneshealth.com.au",
    "healthcareaustralia.com.au",
    "intermed.com.au",
    "health.gov.au",
    "servicesaustralia.gov.au",
    "tga.gov.au",
    "pha.org.au",
    "safetyandquality.gov.au",
    "nhmrc.gov.au",
    "healthshare.nsw.gov.au",
    "austin.org.au",
    "westernhealth.org.au"
]

len(domains)

122

In [ ]:
domain2 = [
'anandaagedcare.com.au',
'aacg.com.au',
'autumncare.com.au',
'basscare.org.au',
'belvedere.com.au',
'bene.org.au',
'bluewaveliving.org.au',
'chomes.com.au',
'columbiaagedcare.com.au',
'fairlea.com.au',
'goodwin.org.au',
'hardiagedcare.com.au',
'havilah.org.au',
'lutheranagedcare.org.au',
'mercycq.com',
'macs.org.au',
'pacnsw.org.au',
'sdagedcare.com.au',
'tlcagedcare.com.au',
'westmont.org.au',
'acan.com.au',
'agedcareconsulting.com.au',
'agedcareexcellence.com.au',
'agedcarefinancesolutions.com.au',
'agedcarehub.com.au',
'agedcareplacement.com.au',
'agedcareready.com.au',
'agedcarespecialists.com.au',
'allthingsagedcare.com.au',
'australianhealthandagedcare.com.au',
'burlingagedcare.com.au',
'familyagedcareadvocates.com.au',
'nextforce.com.au',
'agedcareandmedical.com.au',
'agedcarereviews.com.au',
'findagedcare.com.au',
'flowlogic.com.au',
'opus5k.com.au',
'acetek.com.au',
'acrsmedical.com.au',
'everlightcare.com.au',
'myguardian.com.au',
'paramountcare.com.au',
'youragedcare.com.au',
'acqa.org.au',
'abberfield.com.au',
'agedcarecensus.com.au',
'agedcaregp.com',
'agedcaremadeeasy.com.au',
'agedcaremarketing.com.au',
'agedcarenews.com.au',
'agedcaretrainingservices.com.au',
'agedcarevr.com.au',
'allaboutagedcare.com.au',
'amanagedcare.com.au',
'amberagedcare.com.au',
'ashmangrove.com.au',
'aussieagedcare.com.au',
'aussieagedcare.com.au',
'austcommunityagedcare.com.au',
'autumnlodge.com.au',
'bentleywood.com.au',
'bethelagedcare.com.au',
'boneham.org.au',
'byronagedcare.com.au',
'canberraagedcare.com.au',
'coastalcare.org.au',
'cohs.com.au',
'columbiaagedcare.com.au',
'cbsaust.org.au',
'coorparooagedcare.com.au',
'crowngardens.com.au',
'cura-agedcare.org.au',
'doctoragedcare.com.au',
'dubborslagedcare.com.au',
'eliteagedcare.com.au',
'eac.org.au',
'fairmontagedcare.com.au',
'fairviewagedcare.com.au',
'freewave.com.au',
'goondee.com.au',
'grangeagedcare.com.au',
'greenhillagedcare.com.au',
'hakeagrove.com.au',
'hardengrange.com.au',
'harrisagedcare.com.au',
'holmwood.com.au',
'iacs.org.au',
'isomer.org.au',
'johncurtin.org.au',
'kaagedcare.com.au',
'kewgardens.com.au',
'lithgowagedcare.org.au',
'longridge.org.au',
'manorcourt.org.au',
'maxwellagedcare.com.au',
'mayfieldagedcare.com.au',
'meracare.com.au',
'mercyagedcare.com.au',
'mhas.org.au',
'nanyimaagedcare.com.au',
'navigateagedcare.com.au',
'nestworks.com.au',
'nexia.com.au',
'oakwoodagedcare.com.au',
'osboine.com.au',
'outlookgardens.org.au',
'primeagedcare.com.au',
'profkeagedcare.com.au',
'residencybydillons.com.au',
'rerc.org.au',
'sandringhamgardens.com.au',
'senexus.com.au',
'simplifiedagedcare.com.au',
'sunnymeadepark.com.au',
'tallwoodscorner.com.au',
'thebays.com.au',
'uniqcare.com.au',
'villadalmacia.com.au',
'vincentcourt.com.au',
'warrinaagedcare.com.au',
'westgateagedcare.com.au',
'whyallaagedcare.org.au',
'abbeygardens.com.au',
'acass.com.au',
'adacss.com.au',
'agedcarealternatives.org.au',
'agedcareconnectors.com.au',
'agedcareindustry.com.au',
'agedcaremelbourne.com.au',
'agedcarerelocations.com.au',
'alexanderagedcare.com.au',
'allcareagedcare.com.au',
'amarooagedcare.com.au',
'aquariusagedcare.com.au',
'aspleyagedcare.com.au',
'banfields.com.au',
'bankstownagedcare.com.au',
'bentleys.com.au',
'berriganagedcare.com.au',
'bethanyagedcare.com.au',
'bethelagedcare.com.au',
'brentwoodagedcare.com.au',
'brimleaagedcare.com.au',
'bupa.com.au',
'carinyacare.com.au',
'catholichealthcare.com.au',
'claremontterrace.com.au',
'clarenceestate.com.au',
'cloudagedcare.com.au',
'diac.com.au',
'facebook.com',
'fitzgeraldagedcare.org.au',
'freshfieldsagedcare.com.au',
'gardenview.com.au',
'glendaleagedcare.com.au',
'graftonagedcare.com.au',
'gymeabayagedcare.com.au',
'harbourquaysagedcare.com.au',
'josephbanks.com.au',
'joyly.com.au',
'kewarraagedcare.com.au',
'koonambil.com.au',
'lansdowneagedcare.com.au',
'legacy.com.au',
'macasa.org.au',
'mareebaagedcare.com.au',
'melaleucaagedcare.com.au',
'macds.org.au',
'macn.org.au',
'myvista.com.au',
'nanyimaagedcare.com.au',
'nsacs.org.au',
'narangbaagedcare.com.au',
'oceanmistagedcare.com.au',
'olivegroveagedcare.com.au',
'ourbaysideagedcare.com.au',
'palmsagedcare.com.au',
'pmagedcare.com.au',
'princetonviewagedcare.com.au',
'rainbowcare.org.au',
'rosemoreagedcare.com.au',
'standrewslutheran.org.au',
'stpaulagedcare.com.au',
'theagedcareguru.com.au',
'theagedcarenp.com.au',
'orchardsagedcare.com.au',
'therichardson.com.au',
'agedcare.salvos.org.au',
'twinparks.com.au',
'agedcarefoodsafety.com.au',
'wallsendmanor.com.au',
'waverleyvalley.com.au',
'weeroona.com.au',
'whitehallagedcare.com.au',
'willowbraemelton.com.au',
'wwcac.org.au',
'agedcare.com.au',
'agedcareandmedical.com.au',
'agedcareassistance.com.au',
'acaq.com.au',
'agedcareassessments.com.au',
'acdn.com.au',
'agedcarepathway.com.au',
'agedcarequality.gov.au',
'agedcareseeker.com.au',
'agedcareservicesmelbourne.com.au',
'agedcarespecialistsvic.com.au',
'amberlea.com.au',
'benetas.com.au',
'appliedagedcaresolutions.com.au',
'aacms.com.au',
'aacmc.com.au',
'avonleaagedcare.com.au',
'salvationarmy.org.au',
'baysideagedcare.com.au',
'beechwoodagedcare.com.au',
'hallprior.com.au',
'blakehurstagedcare.com.au',
'catholichealthcare.com.au',
'bridgewateragedcare.com.au',
'calareagedcare.com.au',
'clermontagedcare.com.au',
'coastalwatersagedcare.com.au',
'hallprior.com.au',
'coogeeagedcare.com.au',
'delladaleagedcare.com.au',
'opalhealthcare.com.au',
'donnabrook.com',
'naasa.org.au',
'emeraldterrace.com.au',
'equisent.com.au',
'exactagedcare.com.au',
'fassifernagedcare.com.au',
'floranceagedcare.com.au',
'forcare.com.au',
'geelongagedcare.com.au',
'georginahostel.com.au',
'gleneaglesagedcare.com.au',
'goldstandardagedcare.com.au',
'gpsinagedcare.com.au',
'hcahome.com.au',
'helpnursing.com.au',
'heritagemanor.com.au',
'hibernianagedcare.com.au',
'highwoodcourt.com.au',
'illawarradiggers.com.au',
'illoura.com.au',
'kalicoagedcare.com.au',
'hallprior.com.au',
'latvianagedcare.com.au',
'martynclaveragedcare.com.au',
'hallprior.com.au',
'montclaire.com.au',
'navigatingagedcare.com.au',
'oasisagedcare.com.au',
'orchidagedcare.com.au',
'palmlakeagedcare.com.au',
'parkhillgardens.com.au',
'parkwoodagedcare.com.au',
'quirindiretirementhomes.com.au',
'raen.com.au',
'rmbegg.org.au',
'redleafmanor.com.au',
'resolvemassage.com.au',
'riverwoodagedcare.com.au',
'roseagedcare.com.au',
'stbasilswa.org.au',
'stgeorgeagedcare.com.au',
'strettonpark.com.au',
'sydneyagedcareassist.com.au',
'sydneyagedcaredirectory.com.au',
'sylvanwoods.com.au',
'townandcountryagedcare.com.au',
'trevicourt.com.au',
'universalagedcare.com.au',
'wahroongaagedcare.com.au',
'wecareagedanddisability.com.au',
'westminstervillage.com.au',
'willandraagedcare.com.au',
'windermereagedcare.com.au',
'hallprior.com.au',
'woodsideagedcare.com.au',
'yogaforagedcare.com.au',
'youragedcarecompanion.com.au',
'zenithagedcare.com.au',
'alstonvilleadventistagedcare.com.au',
'appliedmedical.com',
'atosmedical.com',
'avonleagrange.com.au',
'basshillagedcare.com.au',
'byronmedical.com.au',
'anz.medical.canon',
'certafurniture.com.au',
'comfortcare.com.au',
'crkennedy.com.au',
'daltonlodge.com.au',
'devonshirelodge.com.au',
'firstclassagedcare.com.au',
'hallprior.com.au',
'heritagelakes.com.au',
'highwoodcourt.com.au',
'ibdmedical.com.au',
'icumed.com',
'ingeneus.com.au',
'intermedmedical.com.au',
'ipn.com.au',
'jascare.com.au',
'lenexamedical.com',
'macs.org.au',
'medicalaccessories.com.au',
'medicalaesthetic.com.au',
'medsurg.com.au',
'medicalaudiology.com.au',
'medaust.com',
'medicalbillingwa.com.au',
'medicalcareerplanning.com.au',
'mdpp.org.au',
'mehealthcare.com.au',
'medicalequipment.com.au',
'medicalhq.com.au',
'medicalit.com.au',
'medicallearningjourneys.com.au',
'medicalone.com.au',
'msia.com.au',
'mtaa.org.au',
'morphicmedical.com',
'hallprior.com.au',
'multigate.com.au',
'oncoresmedical.com',
'partneredhealth.com.au',
'pendlehillagedcare.com.au',
'pentaxmedical.com',
'hallprior.com.au',
'saludamedical.com',
'samsonmed.com.au',
'secondavenue.com.au',
'senexus.com.au',
'sentrymedical.com.au',
'sirtex.com',
'starkmedical.com.au',
'theoaksagedcare.com.au',
'hallprior.com.au',
'utahmed.com',
'vestech.com.au',
'victoriaboyd.com.au',
'wnl.com.au',
'xtraagedcare.com.au',
'youragedcaresolutions.com.au',
'3dmatrix.com',
'4crisk.com.au',
'armedicalsupplies.com.au',
'activemedical.com.au',
'ams.com.sg',
'akeeko.com',
'anchormedical.com.au',
'anuramedical.com.au',
'arkabamedical.com.au',
'assessmedicalgroup.com.au',
'atma.net.au',
'bettermedical.com.au',
'bnmsafezone.com',
'brams.org.au',
'bydandmedical.com.au',
'candelamedical.com',
'carbal.com.au',
'carilexmedical.com',
'chmc.com.au',
'clarendonmedical.com.au',
'convergencemedical.com',
'detmoldmedical.com',
'diagnosticmedical.com.au',
'elitemedical.com.au',
'emvisionmedical.com',
'equinoxmedical.com.au',
'fairmontmedical.com.au',
'forummedical.com.au',
'gamedical.com.au',
'garnetmedical.com.au',
'gekkomedical.com',
'gpwest.com.au',
'growlifemedical.com.au',
'holdsworthhouse.com.au',
'hydrix.com',
'icaremedicalgroup.com.au',
'iconmedical.com.au',
'imexmedical.com.au',
'imo.com.au',
'imtmedical.com.au',
'inovamedical.com.au',
'intervenemedical.com.au',
'iqmedical.com.au',
'joyamedicalsupplies.com.au',
'jupiterhealth.com.au',
'koalamedical.com.au',
'kprmedical.com.au',
'lateralmedical.com',
'libertymedical.com.au',
'lifemedicalimaging.com.au',
'lumibird-medical.com',
'mmi.org.au',
'macquariemedical.com.au',
'matrixmedical.com.au',
'medclinic.com.au',
'medicross.com.au',
'merit.com',
'minimaximplant.com.au',
'modernmedical.com.au',
'mss-medical.com.au',
'navitechnologies.com',
'nesgroup.com.au',
'norseld.com',
'onehealthclinics.com.au',
'oncosil.com',
'oams.org.au',
'osteonmedical.com',
'puntukurnu.com',
'qheartmedical.com',
'quremed.com.au',
'redhealth.com.au',
'reliancemedical.com.au',
'rocketmedical.com',
'rowemedical.com.au',
'sabremedical.com.au',
'skintech.com.au',
'solta.com',
'sunrisemedical.com.au',
'surgiplasmedical.com.au',
'talemmedical.com.au',
'tcsmedical.com.au',
'teammed.com.au',
'templemedical.com.au',
'medicalcentregympie.com.au',
'medicalnutrition.com.au',
'thrivemedical.com.au',
'tomlinmedical.com.au',
'ultrahealthmedical.com.au',
'vectormedical.com.au',
'ventoramedical.com',
'zonemedical.com.au',
'amsredfern.org.au',
'agilitymedical.com.au',
'ahpdentalmedical.com.au',
'alluremedical.com.au',
'almastreetmedical.com.au',
'angastonmedical.com.au',
'artunismedical.com.au',
'austechmedical.com.au',
'australianmedicalcare.com.au',
'baymedical.com.au',
'beechnutmedical.com.au',
'belmoremedical.com.au',
'bespokemedical.com.au',
'bionicair.com.au',
'boian.com.au',
'bostonscientific.com',
'castlemedical.com.au',
'cochlear.com',
'coloplast.com.au',
'completemedical.com.au',
'edwards.com',
'elitemedical.com.au',
'ellenmedical.com',
'envimedical.com.au',
'fmss.org.au',
'flourishmedical.com.au',
'fusionmedical.com.au',
'gardencitymedical.com.au',
'geelongmedicalgroup.com.au',
'goondiwindimedicalcentre.com.au',
'greenwaymedicalhub.com.au',
'halekulanimedical.com.au',
'healthcareevolutions.com.au',
'healthmint.com.au',
'indigenousmedicalsupplies.com.au',
'isramedical.com.au',
'iupesm2025.org',
'jbmedical.com.au',
'kildareroadmedical.com.au',
'mansemedical.com.au',
'mascotmedical.com.au',
'mediband.com',
'medimac.com.au',
'medline.com.au',
'medme.com.au',
'medpods.com.au',
'monash.edu/mime',
'murraymedical.com.au',
'nimblemedical.com.au',
'nollamaramedical.com.au',
'panaceum.com.au',
'parallelmedical.com.au',
'progressmedical.com.au',
'providencemedical.com.au',
'q1medical.com.au',
'quantummedical.com.au',
'radincon.com.au',
'resolvemedical.com.au',
'ruralmedicalimaging.com.au',
'sanctummedical.com.au',
'sarinamedicalgroup.com.au',
'somi.org.au',
'smsg.com.au',
'stryker.com',
'surmed.com.au',
'talebmedical.com',
'tams.org.au',
'toukleymedical.com.au',
'trinitymedical.com.au',
'uqms.org',
'vausemedical.com.au',
'wellsmedical.com.au',
'workplacemedical.com.au',
'zed3.com.au',
'a2zmedical.com.au',
'abbottmedical.com.au',
'adjmed.com.au',
'airwaymedical.com',
'alphamedical.com.au',
'aquamarinemedical.com.au',
'araratmedicalcentre.com.au',
'archwaymedical.com.au',
'azuremedical.com.au',
'bbraun.com.au',
'baysidemedical.com.au',
'beachendmedical.com.au',
'bluffroadmedical.com.au',
'brightonmedical.com.au',
'chcmedical.com.au',
'clevelandmedical.com.au',
'conmed.com',
'cutera.com',
'dbsmedical.com',
'dexcom.com',
'enovis.com',
'elitemedicalmornington.com.au',
'elitemedicaltoorak.com.au',
'gatewaymedical.com.au',
'geraldtonmedicalgroup.com.au',
'gomedical.com.au',
'greenwaymedicalcentre.com.au',
'hampsteadmedical.com.au',
'hightechmedical.com.au',
'homesteadmedical.com.au',
'iams.org.au',
'kimberleymedicalgroup.com.au',
'lagoonmedicalcentre.com.au',
'lejeunefamily-medical.com.au',
'lockridgemedical.com.au',
'mclinica.com.au',
'mainstreetmedical.com.au',
'metiermedical.com',
'mirandamedicalcentre.com.au',
'modernmedicalclinics.com.au',
'netcaremedical.com.au',
'nhsaustralia.com.au',
'nsmg.com.au',
'onestopmedical.com.au',
'perthmedicalimaging.com.au',
'prioritymedicalcentre.com.au',
'prorad.com.au',
'qutfome.com',
'resmed.com.au',
'richmondmedical.com.au',
'rigicon.com',
'sanaxmedical.com.au',
'skyemedicalarmadale.com.au',
'standrewsmedical.com.au',
'stfrancismedical.com.au',
'stleonardsmedical.com.au',
'toowoombamedicalcentre.com.au',
'tucannmedical.com.au',
'ulladullaendoscopy.com.au',
'verathon.com',
'villagemedicalcentre.com.au',
'westperthmedical.com.au',
'westsidemedicalimaging.com.au',
'whitebridgemedical.com.au',
'wiselinkmedical.com.au',
'yourmedicalhub.com.au',
'zionmedicalcentre.com.au',
'3shape.com',
'abbotsfordmedical.com.au',
'adelaidedisabilitymedical.com.au',
'advancemedicalpractice.com.au',
'almalasers.com',
'alpinemedicalcentre.com.au',
'alsalam.com.au',
'aspiremedicalcentre.com.au',
'atecspine.com',
'avaniaclinical.com',
'banksiamedical.com.au',
'bioumedical.com.au',
'bivacor.com',
'bfmc.com.au',
'bloomfieldmedical.com.au',
'broadbeachmedicalcentre.com.au',
'brookmedical.com.au',
'campsiemedicalpractice.com.au',
'clarencemedical.com.au',
'collingwoodmedical.com.au',
'condaminemc.com.au',
'coringroup.com',
'cosmeticmedicalclinic.com.au',
'craneroadmedical.com.au',
'crinitimedical.com.au',
'darlingtonmedicalcentre.com.au',
'dermasensor.com',
'device.com.au',
'donmedical.com.au',
'duralfamilymedical.com.au',
'fphcare.com',
'getzhealthcare.com',
'gisbornemedical.com.au',
'griffithmedical.com.au',
'hansonmedical.com.au',
'idegroup.com.au',
'innernorthmedical.com.au',
'isimulate.com',
'kiora.com.au',
'laborie.com',
'lifehealthcare.com.au',
'malabarmedical.com.au',
'matravillemedicalcomplex.com.au',
'medel.com',
'medicrew.com.au',
'meridianbioscience.com',
'nanosonics.com.au',
'nsmc.com.au',
'newsteadmedical.com.au',
'nipro-group.com',
'nutromics.com.au',
'onecaremedicalcentre.com.au',
'onyxmedicalcentre.com.au',
'optos.com',
'ovensmedical.com.au',
'paragonmedical.com.au',
'platinummedical.com.au',
'pointblankmedical.com',
'riverstonemedical.com.au',
'salemedical.com.au',
'sensuremedical.com.au',
'southkempseymedical.com.au',
'southsteynemedical.com.au',
'swanmedicalgroup.com.au',
'sydneymedicalspecialists.com.au',
'tenambitmedical.com.au',
'toorakmedicalcentre.com.au',
'torquaymedical.com.au',
'tportho.com',
'wh.com',
'watsoniacentralmedical.com.au',
'westviewmedical.com.au',
'3dicomviewer.com',
'additivesurgical.com',
'baronsmedical.com.au',
'baywestmedical.com.au',
'bedfordmedical.com.au',
'brightmedical.com.au',
'carewellmedical.com.au',
'carlingfordimaging.com.au',
'eastbrookemedical.com.au',
'eastgatemedical.com.au',
'elevatemedical.com.au',
'ellumehealth.com',
'excellamedical.com.au',
'garranmedicalimaging.com.au',
'gasworksmedical.com.au',
'geneabiomedx.com',
'getzhealthcare.com',
'goodnamedical.com.au',
'goulburnmedical.com.au',
'healthemedicals.com.au',
'heine.com',
'illawarramedical.com.au',
'impedimed.com',
'integrativemedicalclinics.com.au',
'kiamamedical.com.au',
'lexmedicus.com.au',
'mayfieldmedicalconnection.com.au',
'meadowsmedical.com.au',
'medline.com.au',
'micro-x.com',
'mlcoa.com.au',
'myhealth.net.au',
'narellanmedical.com.au',
'officermedicalcentre.com.au',
'ormondmc.com.au',
'pandanusmedicalpractice.com.au',
'plazamedical.com.au',
'racma.edu.au',
'robinatownmedicalcentre.com.au',
'sagemedical.com.au',
'salisburymedical.com.au',
'signatureortho.com.au',
'springbankmedical.com.au',
'tewantinmedical.com.au',
'tobwabba.com.au',
'turnthecorner.com.au',
'ufsmedical.com.au',
'ultimomedical.com.au',
'vermontmedicalclinic.com.au',
'vitalitymedicalcentre.com.au',
'waratahmedicalservices.com',
'wiltonmedical.com.au',
'wonthaggimedical.com.au',
'woononamedical.com.au',
'yarramedical.com.au',
'zimmerbiomet.com',
'acemedicalcentre.com.au',
'angleseamedical.com.au',
'auchenflowermedical.com.au',
'bairnsdalemedicalgroup.com.au',
'balmoralmedical.com.au',
'baysidemedical.com.au',
'belvederemedical.com.au',
'berrimedical.com.au',
'blackdoginstitute.org.au',
'bowenhillsmedicalcentre.com.au',
'bridgeviewmedical.com.au',
'brooksidemedical.com.au',
'cammeraymedicalpractice.com.au',
'cannonvalemedicalcentre.com.au',
'carindalemedical.com.au',
'chemtronics.com.au',
'claremontmedical.com.au',
'coffsmedicalcentre.com.au',
'corowamedicalcentre.com.au',
'cowesmedicalcentre.com.au',
'cronullamedical.com.au',
'ebosmedtech.com.au',
'enrichinghealth.com.au',
'eppingmedicalwellness.com.au',
'evolutionmedicalcare.com.au',
'familydoctor.com.au',
'floreymedicalcentre.com.au',
'gamahealthcare.com.au',
'geebungmedical.com.au',
'glengarrymedical.com.au',
'grandmedical.com.au',
'griffinmedical.com.au',
'hartmann.info',
'fairviewgreen.com.au',
'healthtrack.com.au',
'hillsidemedical.com.au',
'icohealth.com.au',
'invacare.com.au',
'jobfit.com.au',
'klsmartin.com',
'marongmedicalpractice.com.au',
'maroubramedical.com.au',
'mckinleymedical.com.au',
'medela.com.au',
'mps.com.au',
'metaoptima.com',
'metromedicalcentre.com.au',
'mingaramedical.com.au',
'moanamedical.com.au',
'mooroolbarkmc.com.au',
'nedlandsmedicalcentre.com.au',
'netacare.com.au',
'omegahealth.com.au',
'painchek.com',
'palettelifesciences.com',
'proteomics.com.au',
'saratogamedical.com.au',
'seascapemedical.com.au',
'supernovamedical.com.au',
'sydneymedicalcentre.com.au',
'tacmedaustralia.com.au',
'toorminamedical.com.au',
'vitalitymedicalcentre.com.au',
'wangarattamedical.com.au',
'waverleymedicalcentre.com.au',
'wellcaremedical.com.au',
'westsidemedical.com.au',
'gabbamedical.com.au',
'zimmerbiomet.com',
'absolutemedicalservices.com.au',
'aldingamedicalcentre.com.au',
'alevia.com.au',
'asquithmedicalcentre.com.au',
'awalmedicalcentre.com.au',
'barrymoremedicalcentre.com.au',
'belgravemedicalclinic.com.au',
'blacktownmedicalcentre.com.au',
'broadwatermedical.com.au',
'burnettmedical.com.au',
'chatswoodmedicaldental.com.au',
'corfieldmedicalcentre.com.au',
'dalkeithmedicalcentre.com.au',
'draytonmedical.com.au',
'duncraigmedicalcentre.com.au',
'eastboundmedical.com.au',
'emeraldmedicalcentre.com.au',
'floreatmedical.com.au',
'garfieldmedicalcentre.com.au',
'resound.com',
'gracemedical.com.au',
'hackhammedical.com.au',
'highwaymedical.com.au',
'hkmedicalcentre.com.au',
'ifamilymedicalcentre.com.au',
'inkermanmedical.com.au',
'ivanhoemedicalclinic.com.au',
'kellyvillemedicalcentre.com.au',
'kelvale.com.au',
'korumburramedical.com.au',
'landsdalemedicalcentre.com.au',
'lithgowmedicalclinic.com.au',
'liverpoolmedicalcentre.com.au',
'macquariemedicalcentre.com.au',
'medacta.com',
'medtronic.com.au',
'mundaringmedical.com.au',
'myhealth.net.au',
'pakenhammedicalclinic.com.au',
'pasadenamedical.com.au',
'polynovo.com',
'pulseimaging.com.au',
'ranzcp.org',
'rehobothmedicalcentres.com.au',
'ripleymedicalskin.com.au',
'riverwaymedical.com.au',
'rutherfordmedical.com.au',
'southcoastmedical.com.au',
'sunburymedicalcentre.com.au',
'unionmedicalcentre.com.au',
'valemedical.com.au',
'vineyardmedicalcentre.com.au',
'wembleymedicalcentre.com.au',
'westonmedical.com.au',
'wyperfeld.com.au',
'121medical.com.au',
'advantagemedical.com.au',
'anatomics.com',
'aspiremedical.com.au',
'ausbiotech.org.au',
'adia.org.au',
'ballartomedicalcentre.com.au',
'beautyfull.com.au',
'bluewatermedical.com.au',
'bridgesidemedical.com.au',
'brisbanemedicalspecialists.com.au',
'bungendoremedicalcentre.com.au',
'byfordmedical.com.au',
'campbelltownmedicaldental.com.au',
'canterburymedical.com.au',
'carindalemedical.com.au',
'caringbahmedicalcentre.com.au',
'carouselmedicalcentre.com.au',
'charltonmedical.com.au',
'crestmeadmedicalcentre.com.au',
'croydonmedical.com.au',
'delacombemedical.com.au',
'dicksonmedical.com.au',
'donnybrookmedical.com.au',
'droversmedicalcentre.com.au',
'edgemedicalcentre.com.au',
'elanmedical.com.au',
'elmwoodmedical.com.au',
'evaluatemedicolegal.com.au',
'gattonmedicalcentre.com.au',
'gracemeremedicalcentre.com.au',
'gungahlinmedicalcentre.com.au',
'headsafe.com',
'hillarysmedical.com.au',
'holmeadroadmedical.com.au',
'humemedical.com.au',
'jamboreemedical.com.au',
'jerrabomberramedicalcentre.com.au',
'jindabynemedical.com.au',
'kadinamedicalassociates.com.au',
'kincraigmedicalclinic.com.au',
'kwinanamedicalcentre.com.au',
'lavertonmedicalcentre.com.au',
'leschenaultmedical.com.au',
'lincolnvillemedical.com.au',
'littlehamptonmedical.com.au',
'majellanmedical.com.au',
'marimamedical.com.au',
'medmont.com',
'medowiemedicalcentre.com.au',
'mediquip.com.au',
'millbankmedical.com.au',
'mundingburramedical.com.au',
'nairnemedical.com.au',
'newingtonmedical.com.au',
'osteonmedical.com',
'plantagenetmedical.com.au',
'polarismedical.com.au',
'primarymedical.com.au',
'queenstreetmedical.com.au',
'qvmedical.com.au',
'ripponleamedical.com.au',
'skyemedical.com.au',
'springmedicalgroup.com.au',
'stirkmedicalgroup.com.au',
'strathdalemedical.com.au',
'theentrancemedicalcentre.com.au',
'treasurymedical.com.au',
'tumbamedical.com.au',
'uniquemedical.com.au',
'uranganmedicalcentre.com.au',
'watervalemedical.com.au',
'wellbeingmedicalcentre.com.au',
'werribeemedical.com.au',
'wingrovemedical.com.au',
'yarrawongamedicalclinic.com.au',
'yerongamedical.com.au',
]

In [ ]:
#@title Bulk Emails - domains

import requests
import pandas as pd
import random
import time

BASE_URL = "https://api.hunter.io/v2/domain-search"

# 🔑 List of Hunter API keys
API_KEYS = []

# 🌐 Input domains
# domains = [
#     "arcare.com.au",
#     "calvarycare.org.au",
#     "boltonclarke.com.au"
# ]

MAX_RETRIES_PER_DOMAIN = 3
REQUEST_SLEEP = 1  # seconds


def extract_hunter_leads(hunter_response):
    """
    Extract structured lead data from Hunter.io domain search response
    """
    data = hunter_response.get("data", {})
    emails = data.get("emails", [])

    company_name = data.get("organization")
    website = data.get("domain")

    leads = []

    for e in emails:
        lead = {
            "company_name": company_name,
            "website": website,
            "full_name": f"{e.get('first_name', '')} {e.get('last_name', '')}".strip(),
            "email": e.get("value"),
            "position": e.get("position"),
            "seniority": e.get("seniority"),
            "department": e.get("department"),
            "linkedin_url": e.get("linkedin"),
            "twitter_url": e.get("twitter"),
            "hunter_confidence": e.get("confidence")
        }
        leads.append(lead)

    return leads


def fetch_domain_leads(domain):
    retries = 0

    while retries < MAX_RETRIES_PER_DOMAIN:
        api_key = random.choice(API_KEYS)

        params = {
            "domain": domain,
            "limit": 10,
            "api_key": api_key
        }

        try:
            response = requests.get(BASE_URL, params=params, timeout=15)

            if response.status_code == 200:
                data = response.json()
                if data.get("data"):
                    print(f"✅ Success: {domain} | API key used: {api_key[-4:]}")
                    return extract_hunter_leads(data)

                print(f"⚠️ Empty data for {domain}")
                return []

            elif response.status_code in [401, 402, 429]:
                print(f"🔁 API key failed ({response.status_code}) → rotating key")

            else:
                print(f"❌ Error {response.status_code} for {domain}")

        except requests.RequestException as e:
            print(f"❌ Request error for {domain}: {e}")

        retries += 1
        time.sleep(REQUEST_SLEEP)

    print(f"🛑 Max retries reached for {domain}")
    return []


if __name__ == "__main__":
    all_leads = []

    for domain in domain2:
        leads = fetch_domain_leads(domain)
        all_leads.extend(leads)
        time.sleep(REQUEST_SLEEP)

    df = pd.DataFrame(all_leads)

    print(f"\n✅ Total leads collected: {len(df)}")
    print(df.head())

    # Optional save
    df.to_csv("hunter_leads_part2.csv", index=False)


✅ Success: anandaagedcare.com.au | API key used: 5f62
✅ Success: aacg.com.au | API key used: 28f7
✅ Success: autumncare.com.au | API key used: 53a7
🔁 API key failed (429) → rotating key
✅ Success: basscare.org.au | API key used: f62c
✅ Success: belvedere.com.au | API key used: 42d0
✅ Success: bene.org.au | API key used: a65e
✅ Success: bluewaveliving.org.au | API key used: d046
✅ Success: chomes.com.au | API key used: 5f62
✅ Success: columbiaagedcare.com.au | API key used: f658
✅ Success: fairlea.com.au | API key used: 96df
✅ Success: goodwin.org.au | API key used: 2914
✅ Success: hardiagedcare.com.au | API key used: bf6f
✅ Success: havilah.org.au | API key used: bf6f
✅ Success: lutheranagedcare.org.au | API key used: 9dfa
✅ Success: mercycq.com | API key used: 1c3b
✅ Success: macs.org.au | API key used: f62c
✅ Success: pacnsw.org.au | API key used: e784
✅ Success: sdagedcare.com.au | API key used: 2914
✅ Success: tlcagedcare.com.au | API key used: 2453
✅ Success: westmont.org.au | API

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
import requests
import time

# List of API Keys to rotate through
API_KEYS = ['1a753bee8b21b9d1927eaf662593c6915bb85409',
 '4f1bb232fe76e58999b32ed7fe108095c29c0f90',
 '948fe0870f7c4c1ced9abd57ac5760e44c81562e',
 '6695991942c29618de3d607388d9398f5d891115',
 'd871ef0e61e2a53e62e3bc4199f043f2b22c9d1c',
 '3cf023b3719386810c03684c96bcf26bcd66a65e',
 '106b5eb9dfd24fa6f0f3ac06b39aa2296c2803fe',
 '4ad138433227e7967feb4cbf761758f8825f69c1',
 '2104bd16df84f03b09a2ff3135c37d2ed810e05a',
 '5daa2ca116eebcb451f0736dc888126983c3f97d',
 'd01b87daaad28550e1d8138ccf1064b103b8ccbc',
 '37c4d39eac8c79af3adcdff263a622c9c3e428d0',
 '79273ba66dd0e4f464c11fdffd6de749783a330d',
 '60ea6716f02ece2e45a33fffcfd299f24041fc4d',
 '1fba2df1900faddb38568b8bfef4f3b18161a15d',
 '0cfd5144a3885dc388d53b452bdf8474ea45542a',
 '91fe1ddad21f8fc3e899ceb48bc41c704ad89269',
 '5b4f5eff7dbc4e5d174af166aca1ea849b0b3f0b',
 '61150cc37813ef999eba7556f301b88e98b12061',
 '8f81a302e9e255841812146dbf99640fb470d53f',
 '663462e543fab55bd97ecaa16c4d81e8f32c53a7',
 'b2bb758b6383cec3f282c4045493af83e8bb06ed',
 '7e1d63f8216ca7bfe85aad42e3649d23bd2ca724',
 'bfd746001b26869f3839d939ea134f345d8c5c3b',
 '0c75c112169e60f02b2a866c22f049492049b278',
 'd51b266149359468a40fac1a090428b4d4f31c3b',
 '12e6bd6da9cfa031ab9fa8270db8dc47db280823',
 '89afbf8e860cc8a9af6d8c66d0bfd0844815af16',
 'dba5e98012e12f5b4b99818e81deb0c06b31e784',
 '8fcc8f81d2ed65d9021a1220d5cf1d97db5cffb6',
 '7a7d02ffb6adb1e8b58b22223b068b8b82ccb44e',
 '8cc4b67acd089c88935f158c304958f423ef4f20',
 '6b80968ffdc2210b33b855a7a405bbb9e12f9dfa',
 '75eb96ded0c6c4d2d716c0531953f5a97ab6f320',
 'ad6c96d31c7e9b4f786af4712fdeb9a3164b3d1d',
 '6d8c752dd2272a5f13a7fd41e13f016227babf6f',
 '1d23c467945ddcf470c6d9d7a8e439515ceb1b7a',
 '9fb5ab9fb73d3d94ef964294279468cc451142d0',
 '08d27200088030162d409cf62fa28797801faa10',
 'e6336dd7189be6c92502cfa17abbf6a9e662f71f',
 '3be4bd56d0e0b0de1b899203b1a98102bede7166',
 '58a9a1e50da5bef69d22801ec1d7c22856a3d046',
 'bf71e5900f4c04e6a8079bb4ed9fc2c828ac10f9',
 '4a028959c2702ca9fe8a4b7bcd098d4014d2d356',
 '1c3e9bc3800aa8b76c2ec5b9e5e9e1ca42194b70',
 'e44e3802ba2d38300ecf1f8d002174e3eab0fd31',
 '9f2f84fb70408e5bf5d84211d4e4287d660428f7',
 '661bef7d159a5343204b4da591444b6a39a72ef7',
 '41feb44cc0e995240d51f2ba146744c229ecf9a3',
 '9f189e87e011a1d2f3013ace7b14045dec60f62c',
 'e8a6c3940e83b75527e8246f584f71866f302453',
 '6e659b94a4b6c71de7fc5f35ab46979048a3059f',
 'e9110a6b18d4becc248f0997a86fa47e8e6f07b1',
 '8257696b56b9d35fe21a3d546e801e635a8b14a7',
 '123a9d5cdb7d8ac5ab7011113b84b75edeefca6f',
 '3771debbaaf7179e7dd429371f9032a69f818979',
 'ad4d5d06aeb363be436e7b1d42724db1765e96df',
 '72167d43207c27e6691b19ab0bbd3a2699b9250d',
 '42cabbf6c8cc2a371ee42414aa7783e500675f62',
 'a6bfa09843516ae31b78089e268c78a47a195bfb',
 '45befd45b616793cbde9cf2142bdc401be79e168',
 '3031fc8992413b35d7b237c6c679859e74d7f658',
 '6a5b4fa748e6bbe572c5470f0f44d92c78ff59aa',
 '3bcc37b4520d497dbc0db86142835349307e79c9',
 'b2e23055abe43d1785d9049e7d100bd8eba04357',
 'f63c142be8f3dfd799fcebef9d429871b1742914',
 '81ce4e6a2bb11717dc61df607bbb7a0c6f7c82ae',
 '2cde17cd082478662ec5f9cf832f4402ad73bc1c',
 '538a63f136a5a2f1e8b812c511197fb8b985539a',
 '8cb6ba1dc4fdf1315f034f39de68f6532e1b1e20',
 '21d82fada636073cdda586efc5fd3151715e5aaa',
 '298345128b334ba3cbf0c4dbf5f6f1f1eecddc22',
 '939aedf77faf87bdef2cf493eb797f2e2fce2c37',
 '85bb5e1b441f3d0993495d6bb8e0ebfe52d5b479',
 '5a34efd85a1ff9d2493919cc95449ce49d74f0ac',
 '2b128fe9b07451039432d4b5d9d4d25580ca901e',
 '5fae6172b8d6f7789a678d33f224a42667e52aaa',
 '0fff49789882a99e01552503036c8fa757680b81',
 '8e5605f59dc9a51f4f50d7bfaac2604bd40aa4c1',
 '0d4701b3e730cbd1b9cd421d358ccbab97f409ca']

BASE_URL = "https://api.hunter.io/v2"

def email_finder_bulk(targets):
    results = []
    key_index = 0  # Start with the first key

    for person in targets:
        name = person.get('name')
        domain = person.get('domain')
        success = False

        while not success:
            if key_index >= len(API_KEYS):
                print("🚨 All API keys have failed or been exhausted!")
                return results

            current_key = API_KEYS[key_index]
            url = f"{BASE_URL}/email-finder"
            params = {
                "full_name": name,
                "domain": domain,
                "api_key": current_key
            }

            print(f"🔎 Searching: {name} @ {domain} (Using Key #{key_index + 1})")

            try:
                response = requests.get(url, params=params)
                res_data = response.json()

                # Check for Authentication or Rate Limit errors
                if "errors" in res_data:
                    error_code = res_data['errors'][0].get('code')
                    error_detail = res_data['errors'][0].get('details')

                    # 401: Invalid Key | 429: Rate Limit Reached
                    if error_code in [401, 429]:
                        print(f"⚠️ Key #{key_index + 1} failed ({error_detail}). Switching key...")
                        key_index += 1
                        continue # Retry the same person with the next key
                    else:
                        print(f"❌ Skipping {name}: {error_detail}")
                        break # Move to next person if it's a data error (e.g. not found)

                # Successful Data Extraction
                data = res_data.get('data', {})
                if data:
                    entry = {
                        "domain": data.get('domain'),
                        "fullname": f"{data.get('first_name', '')} {data.get('last_name', '')}".strip(),
                        "position": data.get('position'),
                        "email": data.get('email'),
                        "score": data.get('score'),
                        "twitter": data.get('twitter'),
                        "linkedin_url": data.get('linkedin_url'),
                        "phone_number": data.get('phone_number')
                    }
                    results.append(entry)

                success = True
                time.sleep(0.5) # Brief pause to be safe

            except Exception as e:
                print(f"⚠️ Connection error: {e}. Retrying...")
                time.sleep(2)

    return results

# --- Example Usage ---
input_list = [
  {"name":"Andy Schmeltz","domain":"csl.com"}
]


final_results = email_finder_bulk(input_list)

# Print Summary
print(f"\n✅ Processed {len(final_results)} records.")
for item in final_results:
    print(item)

In [ ]:
import pandas as pd
df = pd.DataFrame(final_results)
df.to_csv("new_leads.csv", index=False)
df

,domain,fullname,position,email,score,twitter,linkedin_url,phone_number
0,csl.com,Andy Schmeltz,None,andy.schmeltz@csl.com,97.0,None,None,None
1,csl.com,Bill Mezzanotte,None,bill.mezzanotte@csl.com,94.0,None,None,None
2,csl.com,Dave Ross,None,dave.ross@csl.com,98.0,None,None,None
3,csl.com,Greg Boss,"Executive Vice President, General Counsel",greg.boss@csl.com,98.0,None,https://www.linkedin.com/in/gregory-boss,None
4,csl.com,Hervé Gisserot,None,herve.gisserot@csl.com,49.0,None,None,None
...,...,...,...,...,...,...,...,...
180,bupa.com.au,Guneet Sawhney,None,guneet.sawhney@bupa.com.au,95.0,None,None,None
181,bupa.com.au,Nic Woods,None,nic.woods@bupa.com.au,99.0,None,None,None
182,bupa.com.au,Jim Nolan,None,jim.nolan@bupa.com.au,95.0,None,None,None
183,bupa.com.au,Roger Sharp,None,roger.sharp@bupa.com.au,95.0,None,None,None


In [2]:
import requests
import time
import random
from collections import deque

API_KEYS = []

BASE_URL = "https://api.hunter.io/v2"

def email_finder_bulk(targets):
    results = []

    # Shuffle keys once, then rotate
    key_pool = API_KEYS.copy()
    random.shuffle(key_pool)
    keys = deque(key_pool)

    for person in targets:
        name = person.get("name")
        domain = person.get("domain")
        success = False

        while not success:
            if not keys:
                print("🚨 All API keys are invalid or exhausted.")
                return results

            current_key = keys.popleft()  # take next unique key
            url = f"{BASE_URL}/email-finder"
            params = {
                "full_name": name,
                "domain": domain,
                "api_key": current_key
            }

            print(f"🔎 Searching: {name} @ {domain} (Using key ...{current_key[-4:]})")

            try:
                response = requests.get(url, params=params, timeout=15)
                res_data = response.json()

                if "errors" in res_data:
                    error = res_data["errors"][0]
                    code = error.get("code")
                    detail = error.get("details")

                    # Key is bad → discard permanently
                    if code in (401, 429):
                        print(f"⚠️ Key failed ({detail}). Dropping key.")
                        continue  # retry same person with next key

                    # Data-related error → move on
                    print(f"❌ Skipping {name}: {detail}")
                    success = True
                    continue

                # Success
                data = res_data.get("data")
                if data:
                    results.append({
                        "domain": data.get("domain"),
                        "fullname": f"{data.get('first_name', '')} {data.get('last_name', '')}".strip(),
                        "position": data.get("position"),
                        "email": data.get("email"),
                        "score": data.get("score"),
                        "twitter": data.get("twitter"),
                        "linkedin_url": data.get("linkedin_url"),
                        "phone_number": data.get("phone_number")
                    })

                # Key survives → put it back at the end
                keys.append(current_key)
                success = True
                time.sleep(0.5)

            except requests.RequestException as e:
                print(f"⚠️ Network error: {e}. Retrying...")
                keys.append(current_key)
                time.sleep(2)

        # 🔁 When all keys used once → reshuffle
        if len(keys) == len(key_pool):
            random.shuffle(keys)

    return results

# --- Example Usage ---
input_list = [
  {
    "name": "Prabhash Chander Goel",
    "position": "Founder & Managing Director",
    "company_name": "Ananda Aged Care",
    "domain": "anandaagedcare.com.au"
  },
  {
    "name": "Dr. Pooja Newman",
    "position": "Clinical Director",
    "company_name": "Ananda Aged Care",
    "domain": "anandaagedcare.com.au"
  },
  {
    "name": "Michael Page",
    "position": "Workforce Executive / Workforce Development Manager",
    "company_name": "Ananda Aged Care",
    "domain": "anandaagedcare.com.au"
  },
  {
    "name": "Kamal Verma",
    "position": "Hospitality Manager",
    "company_name": "Ananda Aged Care",
    "domain": "anandaagedcare.com.au"
  },
  {
    "name": "Deepti Sudhir",
    "position": "People Performance and Culture Business Partner (HR Manager)",
    "company_name": "Ananda Aged Care",
    "domain": "anandaagedcare.com.au"
  },
  {
    "name": "Karen Daniels",
    "position": "Director of Nursing",
    "company_name": "Ananda Aged Care",
    "domain": "anandaagedcare.com.au"
  },
  {
    "name": "Dermot Clancy",
    "position": "Group CEO",
    "company_name": "AutumnCare (OneTouch Health Group)",
    "domain": "autumn.care"
  },
  {
    "name": "Stuart Sizer",
    "position": "Founder",
    "company_name": "AutumnCare",
    "domain": "autumn.care"
  },
  {
    "name": "Alicia Walker",
    "position": "Sales and Marketing Manager",
    "company_name": "AutumnCare",
    "domain": "autumn.care"
  },
  {
    "name": "Shelley Crowe",
    "position": "Marketing Manager",
    "company_name": "AutumnCare",
    "domain": "autumn.care"
  },
  {
    "name": "Aimee French",
    "position": "Sales and Marketing Coordinator",
    "company_name": "AutumnCare",
    "domain": "autumn.care"
  },
  {
    "name": "Rachel Noel",
    "position": "Director and Owner",
    "company_name": "Belvedere Aged Care",
    "domain": "belvedere.com.au"
  },
  {
    "name": "Marc Noel",
    "position": "Founder (Late Proprietor)",
    "company_name": "Belvedere Aged Care",
    "domain": "belvedere.com.au"
  },
  {
    "name": "Emma Byles",
    "position": "Senior Administration Manager (Marketing & Communications)",
    "company_name": "Belvedere Aged Care",
    "domain": "belvedere.com.au"
  },
  {
    "name": "Helena Johnston",
    "position": "Director of Nursing",
    "company_name": "Belvedere Aged Care",
    "domain": "belvedere.com.au"
  },
  {
    "name": "Nirali Patel",
    "position": "Finance Manager",
    "company_name": "Belvedere Aged Care",
    "domain": "belvedere.com.au"
  },
  {
    "name": "Sharon Rulach",
    "position": "Senior Administration Manager (Facilities and Client Services)",
    "company_name": "Belvedere Aged Care",
    "domain": "belvedere.com.au"
  },
  {
    "name": "Robert Oxford",
    "position": "Chief Executive Officer",
    "company_name": "Hardi Aged Care",
    "domain": "hardiagedcare.com.au"
  },
  {
    "name": "George Hardi",
    "position": "Founder (Late)",
    "company_name": "Hardi Aged Care",
    "domain": "hardiagedcare.com.au"
  },
  {
    "name": "Peter Hardi",
    "position": "Owner / Managing Director",
    "company_name": "Hardi Aged Care",
    "domain": "hardiagedcare.com.au"
  },
  {
    "name": "Jo-Ann Dalton",
    "position": "Chief Operating Officer",
    "company_name": "Hardi Aged Care",
    "domain": "hardiagedcare.com.au"
  },
  {
    "name": "Prilly Wulandari",
    "position": "Executive Finance Manager",
    "company_name": "Hardi Aged Care",
    "domain": "hardiagedcare.com.au"
  },
  {
    "name": "Argie Ramos",
    "position": "Executive Funding Analyst and Business Partner",
    "company_name": "Hardi Aged Care",
    "domain": "hardiagedcare.com.au"
  },
  {
    "name": "Margaret Saltirov",
    "position": "Executive Assets and Administration Manager",
    "company_name": "Hardi Aged Care",
    "domain": "hardiagedcare.com.au"
  },
  {
    "name": "Andrew McFarlane",
    "position": "Chief Executive Officer",
    "company_name": "Bene Aged Care",
    "domain": "bene.org.au"
  },
  {
    "name": "Italian Benevolent Foundation SA",
    "position": "Founder (Established 1974)",
    "company_name": "Bene Aged Care",
    "domain": "bene.org.au"
  },
  {
    "name": "Michael Filsell",
    "position": "Chief Operating Officer",
    "company_name": "Bene Aged Care",
    "domain": "bene.org.au"
  },
  {
    "name": "David Phillips",
    "position": "Chief Financial Officer",
    "company_name": "Bene Aged Care",
    "domain": "bene.org.au"
  },
  {
    "name": "Sunny Traverse",
    "position": "Director of Residential Aged Care",
    "company_name": "Bene Aged Care",
    "domain": "bene.org.au"
  },
  {
    "name": "Catherine McGovern",
    "position": "Manager Quality and Clinical Governance",
    "company_name": "Bene Aged Care",
    "domain": "bene.org.au"
  },
  {
    "name": "Alison Bolton",
    "position": "Chief Executive Officer",
    "company_name": "Christadelphian Aged Care",
    "domain": "chomes.com.au"
  },
  {
    "name": "Ross Peden",
    "position": "Former CEO / Senior Consultant (Retiring July 2026)",
    "company_name": "Christadelphian Aged Care",
    "domain": "chomes.com.au"
  },
  {
    "name": "Phil Cubbin",
    "position": "Chairman of the Board",
    "company_name": "Christadelphian Aged Care",
    "domain": "chomes.com.au"
  },
  {
    "name": "Helen Milanovic",
    "position": "General Manager, Finance",
    "company_name": "Christadelphian Aged Care",
    "domain": "chomes.com.au"
  },
  {
    "name": "Aldrin Garcia",
    "position": "Director, Consumer Care & Services",
    "company_name": "Christadelphian Aged Care",
    "domain": "chomes.com.au"
  },
  {
    "name": "Nick Moutos",
    "position": "Acting Chief Executive Officer / Chief Financial Officer",
    "company_name": "Columbia Aged Care Services",
    "domain": "columbiaagedcare.com.au"
  },
  {
    "name": "Lucy O’Flaherty",
    "position": "Outgoing Chief Executive Officer",
    "company_name": "Columbia Aged Care Services",
    "domain": "columbiaagedcare.com.au"
  },
  {
    "name": "Shelly Silberman",
    "position": "Director",
    "company_name": "Columbia Aged Care Services",
    "domain": "columbiaagedcare.com.au"
  },
  {
    "name": "Sam Arnold",
    "position": "Chief Financial Officer and Company Secretary",
    "company_name": "Columbia Aged Care Services",
    "domain": "columbiaagedcare.com.au"
  },
  {
    "name": "Lizbeth Dela Paz",
    "position": "Care Service Manager",
    "company_name": "Columbia Aged Care Services",
    "domain": "columbiaagedcare.com.au"
  },
  {
    "name": "Mary Ann G. Paringit",
    "position": "Director / Owner",
    "company_name": "Fairlea Aged Care",
    "domain": "fairlea.com.au"
  },
  {
    "name": "Maria Corazon Paringit",
    "position": "Director / Owner",
    "company_name": "Fairlea Aged Care",
    "domain": "fairlea.com.au"
  },
  {
    "name": "John G. Paringit",
    "position": "Director / Owner",
    "company_name": "Fairlea Aged Care",
    "domain": "fairlea.com.au"
  },
  {
    "name": "Lourdes G. Paringit",
    "position": "Director / Owner",
    "company_name": "Fairlea Aged Care",
    "domain": "fairlea.com.au"
  },
  {
    "name": "Madelene M. Paringit",
    "position": "Director / Owner",
    "company_name": "Fairlea Aged Care",
    "domain": "fairlea.com.au"
  },
  {
    "name": "Yousaf Masih",
    "position": "Chief Financial Officer",
    "company_name": "Fairlea Aged Care",
    "domain": "fairlea.com.au"
  },
  {
    "name": "Shubash Thapaliya",
    "position": "Facility Manager / Site Head",
    "company_name": "Fairlea Aged Care",
    "domain": "fairlea.com.au"
  },
  {
    "name": "Tracey Saunders",
    "position": "Chief Executive Officer",
    "company_name": "Havilah Aged Care",
    "domain": "havilah.org.au"
  },
  {
    "name": "Dharmesh Harilal",
    "position": "Chief Financial Officer",
    "company_name": "Havilah Aged Care",
    "domain": "havilah.org.au"
  },
  {
    "name": "Stacey Perry",
    "position": "Director of Care",
    "company_name": "Havilah Aged Care",
    "domain": "havilah.org.au"
  },
  {
    "name": "Jude Aserappa",
    "position": "Executive Director of Resident Experience",
    "company_name": "Havilah Aged Care",
    "domain": "havilah.org.au"
  },
  {
    "name": "Troy Bursill",
    "position": "Manager People and Culture",
    "company_name": "Havilah Aged Care",
    "domain": "havilah.org.au"
  },
  {
    "name": "Danny Tatchell",
    "position": "President of the Board",
    "company_name": "Havilah Aged Care",
    "domain": "havilah.org.au"
  },
  {
    "name": "Wendy Rocks",
    "position": "Chief Executive Officer / Managing Director",
    "company_name": "Lutheran Aged Care Albury",
    "domain": "lacalbury.com"
  },
  {
    "name": "Rachael Scanlon",
    "position": "Chief Financial Officer",
    "company_name": "Lutheran Aged Care Albury",
    "domain": "lacalbury.com"
  },
  {
    "name": "Phillip Edwards",
    "position": "Chairperson of the Board",
    "company_name": "Lutheran Aged Care Albury",
    "domain": "lacalbury.com"
  },
  {
    "name": "Don Elder",
    "position": "Human Resources Manager",
    "company_name": "Lutheran Aged Care Albury",
    "domain": "lacalbury.com"
  },
  {
    "name": "Louise Shields",
    "position": "Organisational Development Manager",
    "company_name": "Lutheran Aged Care Albury",
    "domain": "lacalbury.com"
  },
  {
    "name": "Hellen Ashton",
    "position": "Admissions and Marketing Manager",
    "company_name": "Lutheran Aged Care Albury",
    "domain": "lacalbury.com"
  },
  {
    "name": "Maria Lian Li Johns",
    "position": "Chief Executive Officer",
    "company_name": "Multicultural Aged Care Inc.",
    "domain": "mac.org.au"
  },
  {
    "name": "Italia Mignone",
    "position": "General Manager",
    "company_name": "Multicultural Aged Care Inc.",
    "domain": "mac.org.au"
  },
  {
    "name": "George Genimahaliotis",
    "position": "Chairperson",
    "company_name": "Multicultural Aged Care Inc.",
    "domain": "mac.org.au"
  },
  {
    "name": "Aash Sharma",
    "position": "Communications and Media Officer",
    "company_name": "Multicultural Aged Care Inc.",
    "domain": "mac.org.au"
  },
  {
    "name": "Vandana Rathore",
    "position": "Manager, Programs",
    "company_name": "Multicultural Aged Care Inc.",
    "domain": "mac.org.au"
  },
  {
    "name": "Nigel Fairbairn",
    "position": "Chief Executive Officer",
    "company_name": "Presbyterian Aged Care NSW & ACT",
    "domain": "pacnsw.org.au"
  },
  {
    "name": "The Presbyterian Church in NSW",
    "position": "Founder",
    "company_name": "Presbyterian Aged Care NSW & ACT",
    "domain": "pacnsw.org.au"
  },
  {
    "name": "Margaret See",
    "position": "Chief Financial Officer",
    "company_name": "Presbyterian Aged Care NSW & ACT",
    "domain": "pacnsw.org.au"
  },
  {
    "name": "Lucila Crawford",
    "position": "Chief Operating Officer",
    "company_name": "Presbyterian Aged Care NSW & ACT",
    "domain": "pacnsw.org.au"
  },
  {
    "name": "Andrew de Szoeke",
    "position": "Executive Manager, Property & Business Development",
    "company_name": "Presbyterian Aged Care NSW & ACT",
    "domain": "pacnsw.org.au"
  },
  {
    "name": "Tessa de Szoeke",
    "position": "Head of Marketing & Communications",
    "company_name": "Presbyterian Aged Care NSW & ACT",
    "domain": "pacnsw.org.au"
  },
  {
    "name": "Brian Swanepoel",
    "position": "Chief Executive Officer",
    "company_name": "Seventh-day Adventist Aged Care (Greater Sydney)",
    "domain": "sdaac.com.au"
  },
  {
    "name": "Seventh-day Adventist Church",
    "position": "Founder",
    "company_name": "Seventh-day Adventist Aged Care",
    "domain": "sdaac.com.au"
  },
  {
    "name": "Nadelle Manners",
    "position": "Chief Financial Officer",
    "company_name": "Seventh-day Adventist Aged Care",
    "domain": "sdaac.com.au"
  },
  {
    "name": "Alban Matohiti",
    "position": "President / Board Chair",
    "company_name": "Seventh-day Adventist Aged Care",
    "domain": "sdaac.com.au"
  },
  {
    "name": "Daniel Lavaiamat",
    "position": "Marketing and Communications Manager",
    "company_name": "Seventh-day Adventist Aged Care",
    "domain": "sdaac.com.au"
  },
  {
    "name": "Tony Dunn",
    "position": "Chief Executive Officer",
    "company_name": "Westmont Aged Care Services",
    "domain": "westmont.org.au"
  },
  {
    "name": "Matthew Fagence",
    "position": "Chief Financial Officer",
    "company_name": "Westmont Aged Care Services",
    "domain": "westmont.org.au"
  },
  {
    "name": "Laura Souquet",
    "position": "Director of Care and Clinical Services",
    "company_name": "Westmont Aged Care Services",
    "domain": "westmont.org.au"
  },
  {
    "name": "Maralee Vogel",
    "position": "Director of Community Care",
    "company_name": "Westmont Aged Care Services",
    "domain": "westmont.org.au"
  },
  {
    "name": "Ashley Boyd",
    "position": "Board Chair",
    "company_name": "Westmont Aged Care Services",
    "domain": "westmont.org.au"
  },
  {
    "name": "Angela Collins",
    "position": "Manager Independent & Assisted Living",
    "company_name": "Westmont Aged Care Services",
    "domain": "westmont.org.au"
  },
  {
    "name": "Brian Swanepoel",
    "position": "Managing Director / CEO",
    "company_name": "Adventist Aged Care Sydney",
    "domain": "aacsyd.org.au"
  },
  {
    "name": "Alban Matohiti",
    "position": "President / Communication Director",
    "company_name": "Adventist Aged Care Sydney",
    "domain": "aacsyd.org.au"
  },
  {
    "name": "Nadelle Manners",
    "position": "Chief Financial Officer",
    "company_name": "Adventist Aged Care Sydney",
    "domain": "aacsyd.org.au"
  },
  {
    "name": "Daniel Lavaiamat",
    "position": "Marketing Manager",
    "company_name": "Adventist Aged Care Sydney",
    "domain": "aacsyd.org.au"
  },
  {
    "name": "Ana Paula Andrade",
    "position": "Marketing Coordinator",
    "company_name": "Adventist Aged Care Sydney",
    "domain": "aacsyd.org.au"
  },
  {
    "name": "Pr Terry Johnson",
    "position": "Chairman of the Board",
    "company_name": "Adventist Aged Care Sydney",
    "domain": "aacsyd.org.au"
  },
  {
    "name": "Dr. Rodney J. Manser",
    "position": "Managing Director & Principal Advisor",
    "company_name": "Aged Care Consulting & Advisory Services",
    "domain": "agedcareconsulting.com.au"
  },
  {
    "name": "Kathryne Turek",
    "position": "Founder & Principal Consultant",
    "company_name": "Aged Care Consulting & Advisory Services",
    "domain": "agedcareconsulting.com.au"
  },
  {
    "name": "Nicole J. Manser",
    "position": "Senior Advisor - Dementia Care & Operations",
    "company_name": "Aged Care Consulting & Advisory Services",
    "domain": "agedcareconsulting.com.au"
  },
  {
    "name": "Shaun Ganguly",
    "position": "CEO / Founder / Principal Financial Adviser",
    "company_name": "Aged Care Financial Services",
    "domain": "agedcarefinancialservices.com.au"
  },
  {
    "name": "Mikaela Ganguly",
    "position": "Operations Manager",
    "company_name": "Aged Care Financial Services",
    "domain": "agedcarefinancialservices.com.au"
  },
  {
    "name": "Louisa O'Donovan",
    "position": "Client Relationship Manager",
    "company_name": "Aged Care Financial Services",
    "domain": "agedcarefinancialservices.com.au"
  },
  {
    "name": "Andrew Hopkins",
    "position": "Principal Adviser & Founder",
    "company_name": "WA Aged Care Financial Solutions",
    "domain": "agedcarefswa.com"
  },
  {
    "name": "Tricia Ross",
    "position": "Client Relationship Manager",
    "company_name": "WA Aged Care Financial Solutions",
    "domain": "gedcarefswa.com"
  },
  {
    "name": "Catherine Hopkin",
    "position": "Estate Planning Consultant",
    "company_name": "WA Aged Care Financial Solutions",
    "domain": "agedcarefswa.com"
  },
  {
    "name": "Lyrin Papesh-Brooks",
    "position": "Regional Team Leader",
    "company_name": "Aged Care Placement Services",
    "domain": "agedcareplacement.com.au"
  },
  {
    "name": "Mary Baumann",
    "position": "Chief Executive Officer",
    "company_name": "Aged Care Placement Services",
    "domain": "agedcareplacement.com.au"
  },
  {
    "name": "Catherine (Cath) Chater",
    "position": "CEO / Founder / Principal Consultant",
    "company_name": "Aged Care Ready",
    "domain": "agedcareready.com.au"
  },
  {
    "name": "Siobhan Chater",
    "position": "Client Services & Operations",
    "company_name": "Aged Care Ready",
    "domain": "agedcareready.com.au"
  },
  {
    "name": "Claudia Rigoni-Brazzale",
    "position": "Director & Co-Founder",
    "company_name": "Aged Care Specialists Victoria",
    "domain": "agedcsv.com.au"
  },
  {
    "name": "Angela Wynd",
    "position": "Director & Co-Founder",
    "company_name": "Aged Care Specialists Victoria",
    "domain": "agedcsv.com.au"
  },
  {
    "name": "Sonia Gangi",
    "position": "CEO / Founder",
    "company_name": "All Things Aged Care",
    "domain": "allthingsagedcare.com.au"
  },
  {
    "name": "Sandra Allars",
    "position": "Director & Founder",
    "company_name": "Australian Health & Aged Care",
    "domain": "australianhealthandagedcare.com.au"
  },
  {
    "name": "Tracy Burling",
    "position": "Founder / Inventor",
    "company_name": "Burling Aged Care Solutions",
    "domain": "burlingbeds.com"
  },
  {
    "name": "Anthony Burling",
    "position": "Managing Director",
    "company_name": "Burling Aged Care Solutions",
    "domain": "burlingbeds.com"
  }
]


final_results = email_finder_bulk(input_list)

# Print Summary
print(f"\n✅ Processed {len(final_results)} records.")
for item in final_results:
    print(item)

🔎 Searching: Prabhash Chander Goel @ anandaagedcare.com.au (Using key ...fd31)
🔎 Searching: Dr. Pooja Newman @ anandaagedcare.com.au (Using key ...7166)
🔎 Searching: Michael Page @ anandaagedcare.com.au (Using key ...ccbc)
⚠️ Key failed (Your account was restricted. Please log in to Hunter for more information.). Dropping key.
🔎 Searching: Michael Page @ anandaagedcare.com.au (Using key ...3f0b)
🔎 Searching: Kamal Verma @ anandaagedcare.com.au (Using key ...5409)
🔎 Searching: Deepti Sudhir @ anandaagedcare.com.au (Using key ...69c1)
🔎 Searching: Karen Daniels @ anandaagedcare.com.au (Using key ...f97d)
🔎 Searching: Dermot Clancy @ autumn.care (Using key ...e168)
⚠️ Key failed (You've reached the limit for the number of searches per billing period included in your plan.). Dropping key.
🔎 Searching: Dermot Clancy @ autumn.care (Using key ...14a7)
🔎 Searching: Stuart Sizer @ autumn.care (Using key ...d356)
🔎 Searching: Alicia Walker @ autumn.care (Using key ...2c37)
🔎 Searching: Shelley C

In [3]:
import pandas as pd
df = pd.DataFrame(final_results)
df.to_csv("new_leads_3.csv", index=False)
df

,domain,fullname,position,email,score,twitter,linkedin_url,phone_number
0,anandaagedcare.com.au,None None,None,None,NaN,None,None,None
1,anandaagedcare.com.au,None None,None,None,NaN,None,None,None
2,anandaagedcare.com.au,None None,None,None,NaN,None,None,None
3,anandaagedcare.com.au,None None,None,None,NaN,None,None,None
4,anandaagedcare.com.au,None None,None,None,NaN,None,None,None
...,...,...,...,...,...,...,...,...
101,agedcsv.com.au,None None,None,None,NaN,None,None,None
102,allthingsagedcare.com.au,None None,None,None,NaN,None,None,None
103,australianhealthandagedcare.com.au,None None,None,None,NaN,None,None,None
104,burlingbeds.com,Tracy Burling,None,tracy@burlingbeds.com,95.0,None,None,None
